In [63]:
import vk
import pandas as pd
import random
import time
from sklearn.externals import joblib
from scipy.sparse import csr_matrix
from pathlib import Path
from sklearn.decomposition import TruncatedSVD
from tqdm import tqdm_notebook

## Function to get movies info from VK

In [64]:
def get_data(id, n, min_num_of_movies=3):
    
    movies = pd.Series()
    
    file = Path("id")
    if file.is_file():
        id = joblib.load("id")
        movies = joblib.load("movies")
    last_id = 0
    count = 0
    
    while movies.size < n:
        
        f = vk.get_friends(str(id))
        
        friends = []
        for x in f:
            friends.append(str(x))
        
        user_info = vk.get_users_info(friends)
        
        for friend in user_info.keys():
            if movies.size >= n:
                break
            
            try:
                if count == 100:
                    print("pause")
                    #time.sleep(5)
                    count = 0
                else:
                    count += 1
                    
                

                if ("movies" in user_info[friend].keys() 
                    and len(user_info[friend]["movies"].split(",")) >= min_num_of_movies
                    and not friend in movies.keys()
                    and len(friends) >= 5):
                    if movies.shape[0] % 20 == 0:
                        joblib.dump(movies, filename="movies", compress=9)
                        joblib.dump(friend, filename="id", compress=9)
                    movies.set_value(friend, user_info[friend]["movies"].split(","))
                    print(friend)
            except:
                pass
        print("Index: " + str(movies.shape[0]))
        print("Id: " + str(id))
        print("Last id: " + str(last_id))
        if movies.size < n:
            last_id = id
            while True: 
                ind = random.randint(0, len(friends)-1)
                if (len(vk.get_friends(friends[ind])) >= 5
                    and last_id != friends[ind]):
                    id = friends[ind]
                    break
            
    return movies

## Function to get unique movies

In [65]:
def get_unique_movies(data):
    unique_movies = set()
    for movies in tqdm_notebook(data):
        for movie in movies:        
            unique_movies.add(movie)
    return pd.Series(list(unique_movies))

## Function to construct a matrix

In [66]:
def construct_matrix(data, unique_movies):
    row = []
    column = []

    for user in tqdm_notebook(data.keys()):
        for movie in data[user]:
            row.append(user)
            column.append(unique_movies[unique_movies == movie].index[0])
    ones = [1 for x in range(len(row))]
    
    joblib.dump(row, filename="row", compress=9)
    joblib.dump(column, filename="column", compress=9)
    print(len(row), len(data))

    movies_matrix = csr_matrix((ones, (row, column)), \
                               shape=(len(data), len(unique_movies)))
    
    a = movies_matrix.toarray()
    
    bad_indexes = [x for x in range(len(a))]
    while len(bad_indexes) != 0:
        maxes = a.argmax(1)
        for index in bad_indexes:
            if (a[index][maxes[index]] == 1):
                del bad_indexes[bad_indexes.index(index)]
            else:
                a[index][maxes[index]] = 1
    
    movies_matrix = csr_matrix(a)
    
    return movies_matrix

# Main stuff

In [67]:
file = Path("movies")
if file.is_file():
    data = joblib.load("movies")
else:
    data = get_data(1, 100000, min_num_of_movies=2)

In [68]:
unique_movies = get_unique_movies(data)

In [ ]:
matrix = construct_matrix(data, unique_movies)

In [218]:
svd = TruncatedSVD()
svd.fit(matrix)

TruncatedSVD(algorithm='randomized', n_components=2, n_iter=5,
       random_state=None, tol=0.0)

In [243]:
y = svd.inverse_transform(svd.transform(matrix.todense()[5]))

In [242]:
data[5]

['Ванильное небо',
 ' Дом у озера',
 ' Двухсотлетний человек',
 ' Побег из Шоушенка',
 ' Зеленая миля',
 ' Бой с тенью ...продолжение следует']

In [244]:
for i in range (100):
    print(unique_movies[y.argsort()[0][len(y.argsort())-i]])

 Такси 2
 " "Дворовые танцы"
 2
 3
 Побег из Шоушенка
 Бойцовский клуб
 Зеленая миля
 Достучаться до небес
 4
 Игры разума
 Матрица
 
 Гладиатор
 Начало
 Реквием по мечте
 Властелин колец
 Форест Гамп
 Эффект бабочки
 1+1
 Форрест Гамп
 Список Шиндлера
 Леон
 В погоне за счастьем
 Амели
 Жизнь прекрасна
 Криминальное чтиво
 Назад в будущее
 Вечное сияние чистого разума
 Престиж
 Молчание ягнят
 Спеши любить
 Аватар
 Пианист
 5
 Пролетая над гнездом кукушки
 Титаник
 Секрет
 Крестный отец
 Время
 Отступники
 Семь
 Заплати другому
 Дневник памяти
 Троя
 Ванильное небо
 Иван Васильевич меняет профессию
 Остров проклятых
 Сумерки
 2012
 Один дома
 Служебный роман
 Славные парни
 Трасса 60
 Форсаж
 Куда приводят мечты
 деньги
 Бриллиантовая рука
 Большой куш
 Унесённые призраками
 Иллюзионист
 Поймай меня если сможешь
 12
 Семь жизней
 Мирный воин
 6
 Апокалипсис сегодня
 Остров
 Карты
 Храброе сердце
 Социальная сеть
 99 франков
 Трансформеры
 День сурка
 В джазе только девушки
 Рок-волна


# Personal info recomendation

In [6]:
num_of_blocks = len(data) // 1000 + 1
stuff = len(data) % 1000
users_info = []
for i in tqdm_notebook(range(num_of_blocks)):
    if i == i - 1:
        users_info.append(vk.get_users_info(data.keys()[i*1000:i*1000 + stuff]))
    else:
        users_info.append(vk.get_users_info(data.keys()[i*1000:(i+1)*1000]))

In [ ]:
info = pd.DataFrame(columns=('Sex', 'Age', 'Country', 'Movie', 'Liked'))
for user in tqdm_notebook(users_info[0].keys()):
    sex = users_info[0][user]['sex']
    if sex == 2:
        sex = 0
        
    age = float('NaN')
    if 'bdate' in users_info[0][user].keys():
        ages = users_info[0][user]['bdate'].split(".")
        if len(ages) == 3:
            age = 2016 - int(ages[2])
    
    country = float('NaN')
    if 'country' in users_info[0][user].keys():
        country = users_info[0][user]['country']
    
    movies = set()
    
    for movie in users_info[0][user]['movies'].split(','):
        movies.add(unique_movies[unique_movies == movie].index[0])
    set_len = len(movies)
    good_scores = [1 for x in range(set_len)]
    for i in range(3):
        movies.add(random.randint(0, unique_movies.size - 1))
    bad_scores = [0 for x in range(len(movies)-set_len)]
    scores = []
    scores.extend(good_scores.extend(bad_scores))
    print(len(movies), len(scores))
    

In [47]:
unique_movies[unique_movies == 'Бойцовский клуб'].index[0]

6560